In [1]:
import datetime
import time
import serial
import numpy as np
import pyedflib

In [3]:
# '/dev/ttyUSB0'
# '/dev/ttyACM0'

In [4]:
ser = serial.Serial(
    port = '/dev/ttyUSB0',\
    baudrate=921600
#     parity=serial.PARITY_NONE,\
#     stopbits=serial.STOPBITS_ONE,\
#     bytesize=serial.EIGHTBITS,\
#     timeout=0
)

In [ ]:
now = datetime.datetime.now()
temp_time_stamp = '%02d-%02d-%02d_%02d-%02d' % (now.year, now.month, now.day, now.hour, now.minute)

chennels = 32
edfWriter = pyedflib.EdfWriter(temp_time_stamp+".bdf", chennels, file_type=pyedflib.FILETYPE_BDFPLUS)
channel_info = {'label': 'ch', 'dimension': 'uV', 'sample_rate': int(round(512)),
                'physical_max': 312500, 'physical_min': -312500,
                'digital_max': 8388607, 'digital_min': -8388607,
                'prefilter': 'nome', 'transducer': 'none'}
edfWriter.setTechnician('')
edfWriter.setRecordingAdditional('sleep')
edfWriter.setPatientName('')
edfWriter.setPatientCode('')
edfWriter.setPatientAdditional('')
edfWriter.setAdmincode('')
edfWriter.setEquipment('FreeEEG32')
edfWriter.setGender(0)
edfWriter.setStartdatetime(datetime.datetime.now())

for che in range(chennels):
    edfWriter.setSignalHeader(che, channel_info.copy())
    edfWriter.setLabel(che, 'Channel ' + str(che+1))

frameIndex = 0
data = []
data1 = []
error = []
frameSize = 0

def eeg_null():
    eeg = [0] * 32
    for i in range(32):
        eeg[i] = [] * 1
    return eeg

eeg = eeg_null()
        
while True:
    item = ser.read(ser.inWaiting())
    for x in item:
        #start
        if ( frameIndex == 0 ) and ( x == 160 ):
            frameIndex +=1
        else:
            if ( frameIndex == 1 ):
                frameIndex +=1
                frameSize = 104
            else:
                if ( frameIndex > 1 ) and ( frameIndex < frameSize ):
                    frameIndex +=1
                    data1.append(x)

                else:
                    if ( frameIndex == frameSize ) and ( x == 192 ):
                        if ( len(data1) == 102 ):
                            #eeg
                            for i in range(32):
                                    eeg[i].append(int.from_bytes([data1[3*i], data1[3*i+1], data1[3*i+2]], byteorder = 'big', signed=True))
                            if ( len(eeg[0]) == 512 ):
                                edfWriter.writeSamples(np.asarray(eeg, dtype = np.float64)*0.0372529074)
                                eeg = eeg_null()
                        data1 = []
                        frameIndex = 0
                    else:
#                         print(datetime.datetime.now())

In [7]:
edfWriter.close()